In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from flask import Flask, jsonify
import cx_Oracle

In [4]:
# recipe_point 테이블의 정보를 가져와 csv로 저장 후 csv read
recipe_id_list =[]
a_point_list = []
log_id_list = []

# 전체 선택
dsn=cx_Oracle.makedsn('project-db-stu.ddns.net',1524, 'xe')
conn=cx_Oracle.connect('cgi_2_0516_1/smhrd1@project-db-stu.ddns.net:1524/xe')
cur = conn.cursor()

rs=cur.execute("select * from recipe_point")

for record in rs:
    recipe_id_list.append(record[0])
    a_point_list.append(record[1])
    log_id_list.append(record[2])

conn.commit()
conn.close()

dic = {'recipe_id':recipe_id_list,'a_point':a_point_list,'log_id':log_id_list}
df = pd.DataFrame(dic)
df.to_csv('rating.csv', index ='false', encoding='cp949')

rating = pd.read_csv('rating.csv')
recipe = pd.read_csv('list.csv', encoding = 'cp949')

In [7]:
# rating 데이터와 movies 데이터 결합
rating_recipe = pd.merge(rating, recipe, on='recipe_id')
# 사용자-아이템 평점 행렬 생성
rating_matrix = rating_recipe.pivot_table('a_point','log_id','recipe_id')
# NaN값 0
rating_matrix.fillna(0, inplace=True)
# 아이템-사용자 평점 행렬로 전치
rating_matrix_T = rating_matrix.T
rating_matrix.head(3)

recipe_id,1,2,4,5,6,7,10,11,12,13,...,90941,90960,90980,91023,120344,120401,120441,120463,120476,195453
log_id,,,,,,,,,,,,,,,,,,,,,
chs,3.0,2.0,0.0,4.0,0.0,1.0,5.0,0.0,3.0,2.0,...,0.0,4.0,3.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0
do,3.0,5.0,4.0,5.0,3.0,0.0,0.0,3.0,0.0,3.0,...,2.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0
jj,0.0,0.0,4.0,3.0,5.0,5.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,1.0,3.0,2.0,0.0,5.0,2.0,0.0


In [11]:
# 아이템 유사도 행렬
item_sim = cosine_similarity(rating_matrix_T, rating_matrix_T)

# 데이터 프레임 형태로 저장
item_sim_df = pd.DataFrame(item_sim, index=rating_matrix_T.index, columns=rating_matrix_T.index)

# item_sim_df.shape
item_sim_df.iloc[:4,:4]

recipe_id,1,2,4,5
recipe_id,,,,
1,1.000000,0.649934,0.656532,0.808290
2,0.649934,1.000000,0.487660,0.707594
4,0.656532,0.487660,1.000000,0.560993
5,0.808290,0.707594,0.560993,1.000000


In [22]:
item_sim_df[1].sort_values(ascending=False)[1:6]

recipe_id
281    0.981495
364    0.979912
396    0.944911
331    0.943880
31     0.942809
Name: 1, dtype: float64

In [9]:
# 인수로 사용자-아이템 평점 행렬(NaN은 현재 0으로 대체), 아이템 유사도 행렬 사용
def predict_rating(ratings_arr, item_sim_arr):
    # ratings_arr: u x i, item_sim_arr: i x i
    sum_sr = ratings_arr @ item_sim_arr
    sum_s_abs = np.array( [ np.abs(item_sim_arr).sum(axis=1) ] )
    
    ratings_pred =  sum_sr / sum_s_abs
    
    return ratings_pred

In [12]:
# 사용자별 예측 평점
rating_pred = predict_rating(rating_matrix.values , item_sim_df.values)

rating_pred_matrix = pd.DataFrame(data=rating_pred, index= rating_matrix.index, columns = rating_matrix.columns)
rating_pred_matrix.head(3)

recipe_id,1,2,4,5,6,7,10,11,12,13,...,90941,90960,90980,91023,120344,120401,120441,120463,120476,195453
log_id,,,,,,,,,,,,,,,,,,,,,
chs,1.933480,1.916527,1.355505,1.867674,1.347874,1.658815,2.152681,1.356010,2.489870,2.181998,...,1.324299,2.460858,3.546202,1.424707,1.391949,2.790665,1.471525,1.318137,1.518896,1.469551
do,1.769151,2.747802,1.828385,1.881220,1.764842,1.238538,1.255646,1.988810,1.234553,2.496943,...,2.618800,1.472915,1.207828,1.229196,1.297590,1.220122,1.260317,1.968410,2.075218,1.224336
jj,1.288730,1.284454,1.873791,1.682587,2.136030,3.055600,1.290017,1.792112,1.280114,1.284363,...,1.270497,1.268630,1.283896,1.761980,2.237829,1.993124,1.276469,2.197923,1.630776,1.244596


In [27]:
# 성능 평가는 MSE를 사용
def get_mse(pred, actual):
    # 평점이 있는 실제 영화만 추출 (1차원 배열로 변환)
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    
    return mean_squared_error(pred, actual)

In [29]:
MSE1 = get_mse(rating_pred, rating_matrix.values)
print(f'아이템 기반 모든 인접 이웃 MSE: {MSE1:.4f}')

아이템 기반 모든 인접 이웃 MSE: 2.3785


In [30]:
def predict_rating_topsim(ratings_arr, item_sim_arr, N=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 평점 행렬의 열 크기(아이템 수)만큼 반복 (row: 사용자, col: 아이템)
    for col in range(ratings_arr.shape[1]):
                
        # 특정 아이템의 유사도 행렬 오름차순 정렬시 index .. (1)
        temp = np.argsort(item_sim_arr[:, col]) 
        
        # (1)의 index를 역순으로 나열시 상위 N개의 index = 특정 아이템의 유사도 상위 N개 아이템 index .. (2)
        top_n_items = [ temp[:-1-N:-1] ]
        
        # 개인화된 예측 평점을 계산: 반복당 특정 아이템의 예측 평점(사용자 전체)
        for row in range(ratings_arr.shape[0]):
            
            # (2)의 유사도 행렬
            item_sim_arr_topN = item_sim_arr[col, :][top_n_items].T # N x 1
            
            # (2)의 실제 평점 행렬
            ratings_arr_topN = ratings_arr[row, :][top_n_items]     # 1 x N
            
            # 예측 평점
            pred[row, col] = ratings_arr_topN @ item_sim_arr_topN
            pred[row, col] /= np.sum( np.abs(item_sim_arr_topN) )
            
    return pred

In [31]:
# 사용자별 예측 평점
ratings_pred = predict_rating_topsim(rating_matrix.values , item_sim_df.values, N=20)

# 성능 평가
MSE2 = get_mse(ratings_pred, rating_matrix.values )
print(f'아이템 기반 인접 TOP-20 이웃 MSE: {MSE2:.4f}')

# 예측 평점 데이터 프레임
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= rating_matrix.index,
                                   columns = rating_matrix.columns)

아이템 기반 인접 TOP-20 이웃 MSE: 0.8033


C:\Users\GITCT\AppData\Local\Temp/ipykernel_20508/756703875.py:18: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  item_sim_arr_topN = item_sim_arr[col, :][top_n_items].T # N x 1
C:\Users\GITCT\AppData\Local\Temp/ipykernel_20508/756703875.py:21: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  ratings_arr_topN = ratings_arr[row, :][top_n_items]     # 1 x N


In [33]:
# userId 9가 높은 평점을 준 영화 (실제 평점)
user_rating_id = rating_matrix.loc['chs', :]
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:5]

recipe_id
241    5.0
325    5.0
428    5.0
422    5.0
91     5.0
Name: chs, dtype: float64

In [35]:
# 아직 먹지 않은 레시피 리스트 함수
def get_uneat_recipe(ratings_matrix, userId):
    
    # user_rating: userId의 아이템 평점 정보 (시리즈 형태: title을 index로 가진다.)
    user_rating = ratings_matrix.loc[userId,:]
    
    # user_rating=0인 아직 안먹은 레시피
    uneat_recipe_list = user_rating[ user_rating == 0].index.tolist()
    
    # 모든 레시피명을 list 객체로 만듬. 
    recipe_list = ratings_matrix.columns.tolist()
    
    # 한줄 for + if문으로 안먹은 레시피 리스트 생성
    uneat_list = [ recipe for recipe in recipe_list if recipe in uneat_recipe_list]
    
    return uneat_list


# 먹지 않은 레시피 중 예측 높은 순서로 시리즈 반환
def recomm_recipe_by_userid(pred_df, userId, uneat_list, top_n=10):    
    recomm_recipe = pred_df.loc[userId, uneat_list].sort_values(ascending=False)[:top_n]
    
    return recomm_recipe

In [45]:
# 아직 먹지 않은 레시피 리스트
uneat_list = get_uneat_recipe(rating_matrix, 'chs')

# 아이템 기반의 최근접 이웃 협업 필터링으로 레시피 추천
recomm_recipe = recomm_recipe_by_userid(ratings_pred_matrix, 'chs', uneat_list, top_n=5)

# 데이터 프레임 생성
recomm_recipe = pd.DataFrame(data=recomm_recipe.values, index=recomm_recipe.index, columns=['pred_score'])
recomm_recipe

,pred_score
recipe_id,
362,0.748457
4,0.694542
469,0.491581
272,0.491581
231,0.490856


In [37]:
def recommend(input):
    uneat_list = get_uneat_recipe(rating_matrix, input)

    # 아이템 기반의 최근접 이웃 협업 필터링으로 레시피 추천
    recomm_recipe = recomm_recipe_by_userid(ratings_pred_matrix, input, uneat_list, top_n=10)

    # 데이터 프레임 생성
    recomm_recipe = dict(data=recomm_recipe.values.tolist(), index=recomm_recipe.index.tolist(), columns=['pred_score'])
    return recomm_recipe

In [53]:
s = input('검색할 아이디 ')
temp1= recommend(s)

검색할 아이디 jyp


In [54]:
temp1['index']

[134, 56, 361, 362, 495, 74, 304, 91, 481, 323]

# Flask Server Setting 

In [55]:
import cx_Oracle
from flask import Flask, jsonify,request
import json
from flask_cors import CORS

In [ ]:
app = Flask(__name__)
CORS(app)
@app.route("/",methods =['GET'])
def hello():
    input_id =request.args.get('id')
    print(input_id)
    temp = recommend(input_id)
    return temp

if __name__ == "__main__":
    app.run(host='0.0.0.0', port='8081')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.30.1.32:8081/ (Press CTRL+C to quit)
[2022-06-18 10:30:04,035] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3361, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 76, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 108, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: None

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Prog

None


172.30.1.18 - - [18/Jun/2022 10:31:47] "GET /?id=jj HTTP/1.1" 200 -


jj


...

jyp


172.30.1.18 - - [18/Jun/2022 16:36:46] "GET /?id=jj HTTP/1.1" 200 -


jj


172.30.1.1 - - [18/Jun/2022 19:29:42] "GET /?id=do HTTP/1.1" 200 -


do
